In [1]:
!pip install pyDOE torch matplotlib scipy numpy

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=f8a56f72aca4f767d0ea1a9ba20ef45689d9ede849c4a6dc545fd56b44fbbb71
  Stored in directory: /root/.cache/pip/wheels/96/b9/5d/1138ea8c8f212bce6e97ae58847b7cc323145b3277f2129e2b
Successfully built pyDOE


In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.use('pdf') 
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
import time
import torch
import torch.nn as nn
import torch.optim as optim
from mpl_toolkits.axes_grid1 import make_axes_locatable
import json


np.random.seed(1234)
torch.manual_seed(1234)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class PhysicsInformedNN(nn.Module):
    def __init__(self, layers, lb, ub):
        super(PhysicsInformedNN, self).__init__()
        self.layers = layers
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)

        self.net = nn.Sequential()
        for i in range(len(layers) - 1):
            self.net.add_module(f"linear_{i}", nn.Linear(layers[i], layers[i+1]))
            if i < len(layers) - 2:
                self.net.add_module(f"tanh_{i}", nn.Tanh())

    def forward(self, x):
        return self.net(x)

    def net_uv(self, x, t):
        X = torch.cat([x, t], dim=1)
        X = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        uv = self.forward(X)
        u = uv[:, 0:1]
        v = uv[:, 1:2]

        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, allow_unused=True)[0]
        v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True, allow_unused=True)[0]

        return u, v, u_x, v_x

    def net_f_uv(self, x, t):
        u, v, u_x, v_x = self.net_uv(x, t)

        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True, allow_unused=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True, allow_unused=True)[0]

        v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True, allow_unused=True)[0]
        v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True, allow_unused=True)[0]

        f_u = u_t + 0.5 * v_xx + (u**2 + v**2) * v
        f_v = v_t - 0.5 * u_xx - (u**2 + u**2) * u # This line seems to have a typo, should be v**2
        f_v = v_t - 0.5 * u_xx - (u**2 + v**2) * u


        return f_u, f_v

def train(model, optimizer_adam, optimizer_lbfgs, x0, t0, u0, v0, x_lb, t_lb, x_ub, t_ub, x_f, t_f, niter_adam=20000):
    model = model.to(device)

    x0 = torch.tensor(x0).float().to(device).requires_grad_(True)
    t0 = torch.tensor(t0).float().to(device).requires_grad_(True)
    u0 = torch.tensor(u0).float().to(device)
    v0 = torch.tensor(v0).float().to(device)

    x_lb = torch.tensor(x_lb).float().to(device).requires_grad_(True)
    t_lb = torch.tensor(t_lb).float().to(device).requires_grad_(True)

    x_ub = torch.tensor(x_ub).float().to(device).requires_grad_(True)
    t_ub = torch.tensor(t_ub).float().to(device).requires_grad_(True)

    x_f = torch.tensor(x_f).float().to(device).requires_grad_(True)
    t_f = torch.tensor(t_f).float().to(device).requires_grad_(True)

    def compute_loss():
        u0_pred, v0_pred, _, _ = model.net_uv(x0, t0)
        u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred = model.net_uv(x_lb, t_lb)
        u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = model.net_uv(x_ub, t_ub)
        f_u_pred, f_v_pred = model.net_f_uv(x_f, t_f)

        loss = torch.mean((u0 - u0_pred)**2) + \
               torch.mean((v0 - v0_pred)**2) + \
               torch.mean((u_lb_pred - u_ub_pred)**2) + \
               torch.mean((v_lb_pred - v_ub_pred)**2) + \
               torch.mean((u_x_lb_pred - u_x_ub_pred)**2) + \
               torch.mean((v_x_lb_pred - v_x_ub_pred)**2) + \
               torch.mean(f_u_pred**2) + \
               torch.mean(f_v_pred**2)
        return loss

   
    for it in range(niter_adam):
        optimizer_adam.zero_grad()
        loss = compute_loss()
        loss.backward()
        optimizer_adam.step()
        if it % 100 == 0:
            print(f'Adam Iteration {it}: Loss = {loss.item():.5e}')

   
    def closure():
        optimizer_lbfgs.zero_grad()
        loss = compute_loss()
        loss.backward()
        return loss

    for it in range(20000):  # Max iterations for L-BFGS
        optimizer_lbfgs.step(closure)
        loss = closure()
        if it % 100 == 0:
            print(f'L-BFGS Iteration {it}: Loss = {loss.item():.5e}')

def predict(model, X_star, batch_size=4096):
    model.eval()
    n_points = X_star.shape[0]
    u_pred = np.zeros((n_points, 1))
    v_pred = np.zeros((n_points, 1))
    f_u_pred = np.zeros((n_points, 1))
    f_v_pred = np.zeros((n_points, 1))

    for i in range(0, n_points, batch_size):
        x_batch = torch.tensor(X_star[i:i+batch_size, 0:1]).float().to(device).requires_grad_(True)
        t_batch = torch.tensor(X_star[i:i+batch_size, 1:2]).float().to(device).requires_grad_(True)

        
        u_batch, v_batch, u_x_batch, v_x_batch = model.net_uv(x_batch, t_batch)
        u_t_batch = torch.autograd.grad(u_batch, t_batch, grad_outputs=torch.ones_like(u_batch), create_graph=True, allow_unused=True)[0]
        u_xx_batch = torch.autograd.grad(u_x_batch, x_batch, grad_outputs=torch.ones_like(u_x_batch), create_graph=True, allow_unused=True)[0]
        v_t_batch = torch.autograd.grad(v_batch, t_batch, grad_outputs=torch.ones_like(v_batch), create_graph=True, allow_unused=True)[0]
        v_xx_batch = torch.autograd.grad(v_x_batch, x_batch, grad_outputs=torch.ones_like(v_x_batch), create_graph=True, allow_unused=True)[0]

        
        f_u_batch = u_t_batch + 0.5 * v_xx_batch + (u_batch**2 + v_batch**2) * v_batch
        f_v_batch = v_t_batch - 0.5 * u_xx_batch - (u_batch**2 + v_batch**2) * u_batch

        
        u_pred[i:i+batch_size, :] = u_batch.cpu().detach().numpy()
        v_pred[i:i+batch_size, :] = v_batch.cpu().detach().numpy()
        f_u_pred[i:i+batch_size, :] = f_u_batch.cpu().detach().numpy()
        f_v_pred[i:i+batch_size, :] = f_v_batch.cpu().detach().numpy()

    return u_pred, v_pred, f_u_pred, f_v_pred

if __name__ == "__main__":
    noise = 0.0

    
    lb = np.array([0.0, 0.0])
    ub = np.array([20.0, 5.0])

    N0 = 100
    N_b = 100
    N_f = 20000
    layers = [2, 80, 80, 80, 80, 2]

    data = scipy.io.loadmat('NLS_2soliton_interaction.mat')

    t = data['t_values'].flatten()[:, None]
    x = data['x_values'].flatten()[:, None]
    Exact = data['q_values']

    Exact_u = np.real(Exact)
    Exact_v = np.imag(Exact)
    Exact_h = np.sqrt(Exact_u ** 2 + Exact_v ** 2)

    X, T = np.meshgrid(x, t)

    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    u_star = Exact_u.T.flatten()[:, None]
    v_star = Exact_v.T.flatten()[:, None]
    h_star = Exact_h.T.flatten()[:, None]

    idx_x = np.random.choice(x.shape[0], N0, replace=False)
    x0 = x[idx_x, :]
    u0 = Exact_u[idx_x, 0:1]
    v0 = Exact_v[idx_x, 0:1]

    idx_t = np.random.choice(t.shape[0], N_b, replace=False)
    tb = t[idx_t, :]

    X_f = lb + (ub - lb) * lhs(2, N_f)

    X0 = np.concatenate((x0, 0 * x0), 1)  # (x0, 0)
    X_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
    X_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)
    X_u_train = np.vstack([X0, X_lb, X_ub])

    model = PhysicsInformedNN(layers, lb, ub)
    optimizer_adam = optim.Adam(model.parameters(), lr=0.001)
    optimizer_lbfgs = optim.LBFGS(model.parameters(), max_iter=20, history_size=50, line_search_fn="strong_wolfe")

    start_time = time.time()
    train(model, optimizer_adam, optimizer_lbfgs,
          X0[:, 0:1], X0[:, 1:2], u0, v0,
          X_lb[:, 0:1], X_lb[:, 1:2],
          X_ub[:, 0:1], X_ub[:, 1:2],
          X_f[:, 0:1], X_f[:, 1:2])
    elapsed = time.time() - start_time
    print('Training time: %.4f' % (elapsed))

    
    torch.save(model.state_dict(), 'pytorch_model.pth')
    print("Model saved to 'pytorch_model.pth'")

    u_pred, v_pred, f_u_pred, f_v_pred = predict(model, X_star)
    h_pred = np.sqrt(u_pred ** 2 + v_pred ** 2)

    error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    error_v = np.linalg.norm(v_star - v_pred, 2) / np.linalg.norm(v_star, 2)
    error_h = np.linalg.norm(h_star - h_pred, 2) / np.linalg.norm(h_star, 2)
    print('Error u: %e' % (error_u))
    print('Error v: %e' % (error_v))
    print('Error h: %e' % (error_h))

    U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
    V_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')
    H_pred = griddata(X_star, h_pred.flatten(), (X, T), method='cubic')

    U_star = griddata(X_star, u_star.flatten(), (X, T), method='cubic')
    V_star = griddata(X_star, v_star.flatten(), (X, T), method='cubic')
    H_star = griddata(X_star, h_star.flatten(), (X, T), method='cubic')

    FU_pred = griddata(X_star, f_u_pred.flatten(), (X, T), method='cubic')
    FV_pred = griddata(X_star, f_v_pred.flatten(), (X, T), method='cubic')

    ########################################################################
    
    plt.figure(figsize=(10, 8))
    x_min, x_max = x.min(), x.max()
    cut_indices = np.linspace(0, len(t) - 1, 9, dtype=int)
    y_min, y_max = -0.1, (len(cut_indices) + 1) * 2.0

    for i, idx in enumerate(cut_indices):
        vertical_offset = i * 2.0  # Adjust the spacing between lines as needed
        plt.plot(x, Exact_h[:, idx] + vertical_offset, 'b-', linewidth=1.5, label=f'$t = {t[idx, 0]:.2f}$' if i == 0 else "")
        plt.plot(x, H_pred[idx, :] + vertical_offset, 'r--', linewidth=1.5)
        plt.text(x_min - 1, vertical_offset, f'$t = {t[idx, 0]:.2f}$', fontsize=10, verticalalignment='center')
    plt.xlabel('$x$', fontsize=14)
    plt.ylabel('')
    plt.xlim([x_min, x_max])
    plt.ylim([y_min, y_max])
    plt.gca().set_yticks([])  # Remove y-axis ticks
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend(['Exact', 'Prediction'], loc='upper right', fontsize=10)
    plt.tight_layout()
    plt.savefig('time.pdf', dpi=300)

    ############################ Plotting ###############################
   
    fig, ax = plt.subplots(figsize=(10, 8))

    
    h_img = ax.imshow(H_pred,
                      extent=[lb[0], ub[0], lb[1], ub[1]],  # [x_min, x_max, t_min, t_max]
                      origin='lower',
                      aspect='auto',
                      cmap='YlGnBu')

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(h_img, cax=cax, label='|Q(x, t)|')

    
    ax.plot(X_u_train[:, 0], X_u_train[:, 1],
            'kx', label='Data Points (%d)' % (X_u_train.shape[0]),
            markersize=4, clip_on=False)
    ax.plot(X_f[:, 0], X_f[:, 1],
            'r.', label='Collocation Points (%d)' % (X_f.shape[0]),
            markersize=2)
    ax.legend(loc='upper right', fontsize=10, framealpha=0.8)
    ax.set_xlabel("x", fontsize=14)
    ax.set_ylabel("t", fontsize=14)
    plt.tight_layout()
    plt.savefig('data.pdf', dpi=150)
    plt.show()

    #############################################################################
    plt.figure(figsize=(10, 8))

    # Plotting exact u(t, x)
    plt.subplot(3, 3, 1)
    plt.pcolor(T, X, U_star, cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Exact u(t, x)')
    plt.tight_layout()

    # Plotting exact v(t, x)
    plt.subplot(3, 3, 2)
    plt.pcolor(T, X, V_star, cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Exact v(t, x)')
    plt.tight_layout()

    # Plotting exact h(t, x)
    plt.subplot(3, 3, 3)
    plt.pcolor(T, X, H_star, cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Exact h(t, x)')
    plt.tight_layout()

    # Plotting predicted u(t, x)
    plt.subplot(3, 3, 4)
    plt.pcolor(T, X, U_pred, cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Predicted u(t, x)')
    plt.tight_layout()

    # Plotting predicted v(t, x)
    plt.subplot(3, 3, 5)
    plt.pcolor(T, X, V_pred, cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Predicted v(t, x)')
    plt.tight_layout()

    # Plotting predicted h(t, x)
    plt.subplot(3, 3, 6)
    plt.pcolor(T, X, H_pred, cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Predicted h(t, x)')
    plt.tight_layout()

    # Plotting absolute error for u(t, x)
    plt.subplot(3, 3, 7)
    plt.pcolor(T, X, np.abs(U_star - U_pred), cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Absolute error for u(t, x)')
    plt.text(0.1, 0.9, f'Error u: {error_u:.3e}', color='white', fontsize=10, transform=plt.gca().transAxes)
    plt.tight_layout()

    # Plotting absolute error for v(t, x)
    plt.subplot(3, 3, 8)
    plt.pcolor(T, X, np.abs(V_star - V_pred), cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Absolute error for v(t, x)')
    plt.text(0.1, 0.9, f'Error v: {error_v:.3e}', color='white', fontsize=10, transform=plt.gca().transAxes)
    plt.tight_layout()

    # Plotting absolute error for h(t, x)
    plt.subplot(3, 3, 9)
    plt.pcolor(T, X, np.abs(H_star - H_pred), cmap='gray')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Absolute error for h(t, x)')
    plt.text(0.1, 0.9, f'Error h: {error_h:.3e}', color='white', fontsize=10, transform=plt.gca().transAxes)
    plt.tight_layout()
    plt.savefig('U_V_H_errors.pdf', dpi=25, bbox_inches='tight')

    np.savez_compressed('PINN_results.npz',
        x=X[0, :],                  # x-axis
        t=T[:, 0],                  # t-axis
        X=X,
        T=T,
        Exact_h = Exact_h,
        U_star=U_star,
        V_star=V_star,
        H_star=H_star,
        U_pred=U_pred,
        V_pred=V_pred,
        H_pred=H_pred,
        X_u_train=X_u_train,
        X_f=X_f,
        lb=lb,
        ub=ub,
        error_u=error_u,
        error_v=error_v,
        error_h=error_h
    )

    metadata = {
        'layers': layers,
        'N0': int(N0),
        'Nb': int(N_b),
        'Nf': int(N_f),
        'lb': lb.tolist(),
        'ub': ub.tolist(),
        'error_u': float(error_u),
        'error_v': float(error_v),
        'error_h': float(error_h),
        'training_time_sec': float(elapsed)
    }

    with open('PINN_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=4)

    
    fig_anim, ax_anim = plt.subplots(figsize=(10, 6))
    x_flat = x.flatten()  # Ensure x is 1D
    t_flat = t.flatten()  # Ensure t is 1D

    
    line_exact, = ax_anim.plot(x_flat, H_star[0, :], 'b-', label='Exact', lw=2)
    line_pred, = ax_anim.plot(x_flat, H_pred[0, :], 'r--', label='PINN', lw=2)

   
    ax_anim.set_xlim(x_flat.min(), x_flat.max())
    ax_anim.set_ylim(-0.5, H_star.max() + 0.5)  # Adjust ylim for magnitude (non-negative)
    ax_anim.set_title('Evolución |q(x,t)|')
    ax_anim.set_xlabel('x')
    ax_anim.set_ylabel('|q|')
    ax_anim.legend(loc='upper right')
    ax_anim.grid(True)

    def animate(i):
        line_exact.set_ydata(H_star[i, :])
        line_pred.set_ydata(H_pred[i, :])
        ax_anim.set_title(f'Evolución |q(x,t={t_flat[i]:.2f})')
        return line_exact, line_pred

    anim = FuncAnimation(fig_anim, animate, frames=len(t_flat), interval=100, blit=False, repeat=True)
    anim.save('nls_evolution.gif', writer='pillow', fps=10)
    plt.show()  

Using device: cuda
Adam Iteration 0: Loss = 2.18551e-01
Adam Iteration 100: Loss = 1.85658e-01
Adam Iteration 200: Loss = 1.85010e-01
Adam Iteration 300: Loss = 1.83586e-01
Adam Iteration 400: Loss = 1.80091e-01
Adam Iteration 500: Loss = 1.77886e-01
Adam Iteration 600: Loss = 1.54663e-01
Adam Iteration 700: Loss = 1.40203e-01
Adam Iteration 800: Loss = 1.29077e-01
Adam Iteration 900: Loss = 1.20391e-01
Adam Iteration 1000: Loss = 1.12479e-01
Adam Iteration 1100: Loss = 1.04172e-01
Adam Iteration 1200: Loss = 9.73107e-02
Adam Iteration 1300: Loss = 9.92787e-02
Adam Iteration 1400: Loss = 8.86948e-02
Adam Iteration 1500: Loss = 8.55556e-02
Adam Iteration 1600: Loss = 8.07710e-02
Adam Iteration 1700: Loss = 7.86055e-02
Adam Iteration 1800: Loss = 7.72135e-02
Adam Iteration 1900: Loss = 7.63640e-02
Adam Iteration 2000: Loss = 7.58664e-02
Adam Iteration 2100: Loss = 7.51727e-02
Adam Iteration 2200: Loss = 7.48060e-02
Adam Iteration 2300: Loss = 7.45166e-02
Adam Iteration 2400: Loss = 7.425